In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertTokenizer, BertModel

In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\Application\Python\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Prime\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [3]:
def preprocess(text):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    return inputs

In [4]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.fc = nn.Linear(self.bert.config.hidden_size, 1)

    def forward_once(self, x):
        outputs = self.bert(**x)
        pooled_output = outputs.pooler_output
        return pooled_output

    def forward(self, x1, x2):
        output1 = self.forward_once(x1)
        output2 = self.forward_once(x2)
        return output1, output2

In [5]:
def train_siamese_network(siamese_net, data_loader, criterion, optimizer, num_epochs=10):
    siamese_net.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for batch in data_loader:
            optimizer.zero_grad()
            inputs1, inputs2, labels = batch
            output1, output2 = siamese_net(inputs1, inputs2)
            loss = criterion(output1, output2, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs1.size(0)
        epoch_loss = running_loss / len(data_loader.dataset)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss}")

In [6]:
def compute_similarity(siamese_net, text1, text2):
    inputs1 = preprocess(text1)
    inputs2 = preprocess(text2)
    output1, output2 = siamese_net(inputs1, inputs2)
    similarity_score = nn.functional.cosine_similarity(output1, output2)
    return similarity_score.item()

In [7]:
siamese_net = SiameseNetwork()

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [8]:
criterion = nn.CosineEmbeddingLoss()
optimizer = optim.AdamW(siamese_net.parameters(), lr=1e-5)

In [9]:
text1 = "This is the first article."
text2 = "This is another article."
similarity = compute_similarity(siamese_net, text1, text2)
print(f"Similarity between the articles: {similarity}")

Similarity between the articles: 0.9955118894577026
